In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
input_dir = "/content/drive/MyDrive/IOT/SoundClasification/Processed_Dataset"
output_dir = "/content/drive/MyDrive/IOT/SoundClasification/AgumentedData"

In [10]:
import os
import librosa
import numpy as np
import soundfile as sf
import random

# Parameters
AUGMENTATIONS_PER_FILE = 5  # Apply all augmentations (5 augmentations)
TARGET_SR = 16000  # Fixed sample rate of 16,000 Hz

# Paths
# input_dir = "input_folder"
# output_dir = "output_folder"

# Augmentation Functions
def pitch_shift(y, sr):
    n_steps = random.uniform(-2, 2)  # Random pitch shift between -2 and 2 semitones
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def add_noise(y, noise_level=0.005):
    noise = np.random.randn(len(y))  # Generate random noise
    return y + noise_level * noise  # Add noise to original audio

def time_shift(y):
    shift = int(random.uniform(-0.1, 0.1) * len(y))  # Time shift between -10% and 10%
    return np.roll(y, shift)  # Apply the time shift

def change_volume(y):
    gain_db = random.uniform(-6, 6)  # Random volume gain between -6 and +6 dB
    return y * (10 ** (gain_db / 20))  # Apply volume scaling

# Ensure that all augmented outputs are of fixed length (same as original audio length)
def enforce_length(y, target_len):
    # If the length is less, pad it. If greater, trim it.
    if len(y) < target_len:
        return np.pad(y, (0, target_len - len(y)))  # Padding to target length
    return y[:target_len]  # Cropping to target length

# Apply all augmentations
def apply_augmentations_all(y, sr):
    y1 = pitch_shift(np.copy(y), sr)
    y2 = add_noise(np.copy(y))
    y3 = time_shift(np.copy(y))
    y4 = change_volume(np.copy(y))
    return [("pitch_shift", y1), ("add_noise", y2), ("time_shift", y3),
            ("change_volume", y4)]

# Processing the dataset
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if not file.lower().endswith(".wav"):  # Skip non-WAV files
            continue

        class_folder = os.path.relpath(root, input_dir)  # Get subfolder name (class)
        output_class_dir = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)  # Create class folder in output

        file_path = os.path.join(root, file)
        y, sr = librosa.load(file_path, sr=TARGET_SR)  # Load audio with target sample rate
        original_length = len(y)  # Get the length of the original audio

        augmented_list = apply_augmentations_all(y, sr)  # Apply all augmentations
        for aug_name, y_aug in augmented_list:
            y_aug = enforce_length(y_aug, original_length)  # Ensure augmented audio is same length as original
            save_path = os.path.join(output_class_dir, f"{os.path.splitext(file)[0]}_{aug_name}.wav")
            sf.write(save_path, y_aug, sr)  # Save augmented file

print("Augmentation completed.")


Augmentation completed.
